# Capstone Project


### Note: This notebook will be used for the capstone project required for the IBM Data Science Specialization on coursera.

The goal of this notebook is, to cluster the neighborhoods of munich in order to find similar neighborhoods to live in.

In [2]:
# necessary imports
import pandas as pd
import numpy as np

In [3]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Let's get the geographical coordinates of Munich.

In [40]:
address = 'Munich'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude_munich = location.latitude
longitude_munich = location.longitude
print('The geograpical coordinate of Munich are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Munich are 48.1149632, 11.5701894.


Let's get the postal codes of munich in germany.

In [6]:
url = 'https://www.muenchen.de/int/en/living/postal-codes.html'
munich_data_list = pd.read_html(url)
munich_data = munich_data_list[0]
munich_data

,District,Postal Code
0,Allach-Untermenzing,"80995, 80997, 80999, 81247, 81249"
1,Altstadt-Lehel,"80331, 80333, 80335, 80336, 80469, 80538, 80539"
2,Au-Haidhausen,"81541, 81543, 81667, 81669, 81671, 81675, 81677"
3,Aubing-Lochhausen-Langwied,"81243, 81245, 81249"
4,Berg am Laim,"81671, 81673, 81735, 81825"
5,Bogenhausen,"81675, 81677, 81679, 81925, 81927, 81929"
6,Feldmoching-Hasenbergl,"80933, 80935, 80995"
7,Hadern,"80689, 81375, 81377"
8,Laim,"80686, 80687, 80689"
9,Ludwigsvorstadt-Isarvorstadt,"80335, 80336, 80337, 80469"


## Preprocessing

Let's start to preprocess the data in order to have a acceptable data format

First step: Split all places according to their postal codes

In [7]:
munich_data_cleaned = pd.DataFrame(columns=['District', 'Postal Code'])
munich_data_cleaned.head()

,District,Postal Code


In [8]:
items = []
for idx, codes in enumerate(munich_data['Postal Code']):
    code_list = codes.split(',')
    district = munich_data['District'][idx]
    for element in code_list:
        element = element.replace(' ', '')
        items.append({'District': district, 'Postal Code': element})

In [9]:
munich_data_cleaned = munich_data_cleaned.append(items)
munich_data_cleaned.head()

,District,Postal Code
0,Allach-Untermenzing,80995
1,Allach-Untermenzing,80997
2,Allach-Untermenzing,80999
3,Allach-Untermenzing,81247
4,Allach-Untermenzing,81249


Let's now fetch all latitude and longitude values for each Postal Code by using the Foursquare API

In [10]:
# credentials
CLIENT_ID = 'S3SWDZ4KH1KEXMKCPNHUQT0HDXWF2YHNZMYNE4UCNK0O2G1C' # your Foursquare ID
CLIENT_SECRET = 'PKGFEDIOKLA3RWYA5EY0SM2WDWU42MCTJPO1D012L2JVI1BO' # your Foursquare Secret
VERSION = '20200410' # Foursquare API version
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: S3SWDZ4KH1KEXMKCPNHUQT0HDXWF2YHNZMYNE4UCNK0O2G1C
CLIENT_SECRET:PKGFEDIOKLA3RWYA5EY0SM2WDWU42MCTJPO1D012L2JVI1BO


In [11]:
# create new dataframe additionally containing the latitude and longitude values of each district and postal code mapping 
munich_data_ll = pd.DataFrame(columns=['District', 'Postal Code', 'Latitude', 'Longitude'])

# loop over all entries of old data frame and store according values
items = []
for idx, district in enumerate(munich_data_cleaned['District']):
    code = munich_data_cleaned['Postal Code'][idx]
    address = district + ', ' + code # to get format of address

    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    items.append({'District': district, 
                  'Postal Code': code,
                  'Latitude': latitude,
                  'Longitude': longitude})

In [12]:
munich_data_ll = munich_data_ll.append(items)
munich_data_ll.head()

,District,Postal Code,Latitude,Longitude
0,Allach-Untermenzing,80995,48.190034,11.468105
1,Allach-Untermenzing,80997,48.183434,11.488162
2,Allach-Untermenzing,80999,48.195994,11.457013
3,Allach-Untermenzing,81247,48.176884,11.476058
4,Allach-Untermenzing,81249,48.176884,11.476058


## Visualization

Let's now create a map of all districts in munich.

In [13]:
# create map of munich using latitude and longitude values
map_munich = folium.Map(location=[munich_data_ll["Latitude"].iloc[0], munich_data_ll["Longitude"].iloc[0]], zoom_start=11)

# add markers to map
for lat, lng, district in zip(munich_data_ll['Latitude'], munich_data_ll['Longitude'], munich_data_ll['District']):
    label = '{}'.format(district)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_munich)  
    
map_munich

## District Exploration

Let's now explore all districts in munich by fetching venues in the near of each district with the help of the foursquare API.

In [16]:
# function for getting all venues of munich
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
munich_venues = getNearbyVenues(names=munich_data_ll['District'],
                                   latitudes=munich_data_ll['Latitude'],
                                   longitudes=munich_data_ll['Longitude']
                                  )

Allach-Untermenzing
Allach-Untermenzing
Allach-Untermenzing
Allach-Untermenzing
Allach-Untermenzing
Altstadt-Lehel
Altstadt-Lehel
Altstadt-Lehel
Altstadt-Lehel
Altstadt-Lehel
Altstadt-Lehel
Altstadt-Lehel
Au-Haidhausen
Au-Haidhausen
Au-Haidhausen
Au-Haidhausen
Au-Haidhausen
Au-Haidhausen
Au-Haidhausen
Aubing-Lochhausen-Langwied
Aubing-Lochhausen-Langwied
Aubing-Lochhausen-Langwied
Berg am Laim
Berg am Laim
Berg am Laim
Berg am Laim
Bogenhausen
Bogenhausen
Bogenhausen
Bogenhausen
Bogenhausen
Bogenhausen
Feldmoching-Hasenbergl
Feldmoching-Hasenbergl
Feldmoching-Hasenbergl
Hadern
Hadern
Hadern
Laim
Laim
Laim
Ludwigsvorstadt-Isarvorstadt
Ludwigsvorstadt-Isarvorstadt
Ludwigsvorstadt-Isarvorstadt
Ludwigsvorstadt-Isarvorstadt
Maxvorstadt
Maxvorstadt
Maxvorstadt
Maxvorstadt
Maxvorstadt
Maxvorstadt
Maxvorstadt
Maxvorstadt
Maxvorstadt
Milbertshofen-Am Hart
Milbertshofen-Am Hart
Milbertshofen-Am Hart
Milbertshofen-Am Hart
Moosach
Moosach
Moosach
Moosach
Moosach
Neuhausen-Nymphenburg
Neuhausen-Nym

In [18]:
# lets get the shape of the new dataframe
munich_venues.shape

(3566, 7)

In [19]:
# Lets visualize the head of the new dataframe
munich_venues.head()

,District,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Allach-Untermenzing,48.190034,11.468105,Sport Bittl,48.191447,11.466553,Sporting Goods Shop
1,Allach-Untermenzing,48.190034,11.468105,Trattoria Olive,48.189905,11.466970,Trattoria/Osteria
2,Allach-Untermenzing,48.190034,11.468105,dm-drogerie markt,48.194118,11.465640,Drugstore
3,Allach-Untermenzing,48.190034,11.468105,Rossmann,48.193301,11.466388,Drugstore
4,Allach-Untermenzing,48.190034,11.468105,Hotel im Bunker,48.189208,11.466852,Hotel


Let's check how many venues were returned for each district

In [20]:
munich_venues.groupby('District').count()

,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
District,,,,,,
Allach-Untermenzing,35,35,35,35,35,35
Altstadt-Lehel,623,623,623,623,623,623
Au-Haidhausen,259,259,259,259,259,259
Aubing-Lochhausen-Langwied,21,21,21,21,21,21
Berg am Laim,40,40,40,40,40,40
Bogenhausen,42,42,42,42,42,42
Feldmoching-Hasenbergl,15,15,15,15,15,15
Hadern,39,39,39,39,39,39
Laim,67,67,67,67,67,67


Let's find out how many unique categories can be curated from all the returned venues

In [21]:
print('There are {} uniques categories.'.format(len(munich_venues['Venue Category'].unique())))

There are 199 uniques categories.


## Analyze each District

Now lets analyze each district in order to get an idea of venues in the districts.

In [26]:
# lets get a one hot encoding of all differen
munich_onehot = pd.get_dummies(munich_venues[['Venue Category']], prefix="", prefix_sep="")

# add District column to dataframe
munich_onehot.insert(0, 'District', munich_data_ll['District'])
munich_onehot.head()

,District,Accessories Store,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Dealership,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Bavarian Restaurant,Beach,Beer Bar,Beer Garden,Bistro,Boat Rental,Bookstore,Boutique,Brewery,Bridge,Burger Joint,Burrito Place,Bus Line,Bus Stop,Café,Canal,Candy Store,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,Gastropub,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hawaiian Restaurant,Historic Site,History Museum,Hookah Bar,Hostel,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Internet Cafe,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Lake,Laundromat,Laundry Service,Lebanese Restaurant,Light Rail Station,Liquor Store,Lottery Retailer,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Motel,Mountain,Movie Theater,Museum,Music Venue,Newsstand,Nightclub,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Pool,Post Office,Print Shop,Pub,Public Art,Ramen Restaurant,Rental Car Location,Restaurant,River,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Science Museum,Seafood Restaurant,Shipping Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soup Place,Spa,Spanish Restaurant,Sporting Goods Shop,Stadium,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Tapas Restaurant,Taverna,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tour Provider,Toy / Game Store,Trail,Train Station,Tram Station,Trattoria/Osteria,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Wine Bar,Wine Shop,Xinjiang Restaurant,Yoga Studio
0,Allach-Untermenzing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Allach-Untermenzing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,Allach-Untermenzing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Allach-Untermenzing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [27]:
munich_grouped = munich_onehot.groupby('District').mean().reset_index()
munich_grouped.head(10)

,District,Accessories Store,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Dealership,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Bavarian Restaurant,Beach,Beer Bar,Beer Garden,Bistro,Boat Rental,Bookstore,Boutique,Brewery,Bridge,Burger Joint,Burrito Place,Bus Line,Bus Stop,Café,Canal,Candy Store,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,Gastropub,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hawaiian Restaurant,Historic Site,History Museum,Hookah Bar,Hostel,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Internet Cafe,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Lake,Laundromat,Laundry Service,Lebanese Restaurant,Light Rail Station,Liquor Store,Lottery Retailer,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Motel,Mountain,Movie Theater,Museum,Music Venue,Newsstand,Nightclub,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Pool,Post Office,Print Shop,Pub,Public Art,Ramen Restaurant,Rental Car Location,Restaurant,River,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Science Museum,Seafood Restaurant,Shipping Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soup Place,Spa,Spanish Restaurant,Sporting Goods Shop,Stadium,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Tapas Restaurant,Taverna,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tour Provider,Toy / Game Store,Trail,Train Station,Tram Station,Trattoria/Osteria,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Wine Bar,Wine Shop,Xinjiang Restaurant,Yoga Studio
0,Allach-Untermenzing,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.200000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.20,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,Altstadt-Lehel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.000000

Let's check the new size

In [28]:
munich_grouped.shape

(25, 200)

#### Let's print each neighborhood along with the top 5 most common venues

In [29]:
num_top_venues = 5

for hood in munich_grouped['District']:
    print("----"+hood+"----")
    temp = munich_grouped[munich_grouped['District'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allach-Untermenzing----
                 venue  freq
0            Drugstore   0.4
1  Sporting Goods Shop   0.2
2                Hotel   0.2
3    Trattoria/Osteria   0.2
4          Pastry Shop   0.0


----Altstadt-Lehel----
                   venue  freq
0            Supermarket  0.29
1               Bus Stop  0.14
2    Sporting Goods Shop  0.14
3  Vietnamese Restaurant  0.14
4     Light Rail Station  0.14


----Au-Haidhausen----
                venue  freq
0              Bakery  0.29
1            Bus Stop  0.29
2  Turkish Restaurant  0.14
3      Cosmetics Shop  0.14
4   Food & Drink Shop  0.14


----Aubing-Lochhausen-Langwied----
                 venue  freq
0  Bavarian Restaurant  0.67
1   Italian Restaurant  0.33
2    Accessories Store  0.00
3          Pastry Shop  0.00
4            Nightclub  0.00


----Berg am Laim----
               venue  freq
0           Bus Stop  0.25
1          Gift Shop  0.25
2       Tennis Court  0.25
3  Trattoria/Osteria  0.25
4  Accessories Store  0.00

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:] # exclude District column
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [53]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['District']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
district_venues_sorted = pd.DataFrame(columns=columns)
district_venues_sorted['District'] = munich_grouped['District']

for ind in np.arange(munich_grouped.shape[0]):
    district_venues_sorted.iloc[ind, 1:] = return_most_common_venues(munich_grouped.iloc[ind, :], num_top_venues)

district_venues_sorted.head()

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allach-Untermenzing,Drugstore,Hotel,Trattoria/Osteria,Sporting Goods Shop,Yoga Studio,Farmers Market,Fountain,Food Court,Food & Drink Shop,Food
1,Altstadt-Lehel,Supermarket,Sporting Goods Shop,Vietnamese Restaurant,Bus Stop,Light Rail Station,Bakery,Yoga Studio,Fast Food Restaurant,Fountain,Food Court
2,Au-Haidhausen,Bakery,Bus Stop,Cosmetics Shop,Turkish Restaurant,Food & Drink Shop,Yoga Studio,French Restaurant,Fountain,Food Court,Food
3,Aubing-Lochhausen-Langwied,Bavarian Restaurant,Italian Restaurant,Fast Food Restaurant,French Restaurant,Fountain,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market
4,Berg am Laim,Bus Stop,Gift Shop,Trattoria/Osteria,Tennis Court,Yoga Studio,Fast Food Restaurant,Fountain,Food Court,Food & Drink Shop,Food


## Cluster Neighborhoods

Now that we have an overview about the data and made some first explorations, it's time to cluster the neighborhoods in order to get an idea about the types of neighborhoods and which district seems to be similar to which other districts.

In [54]:
num_clusters = 5

X = munich_grouped.drop('District', 1)

kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(X)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [55]:
# add clustering labels
district_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

munich_merged = munich_data_ll

# merge labels and data about venues to district data and latitude plus longitude data to have all in one dataframe
munich_merged = munich_merged.join(district_venues_sorted.set_index('District'), on='District')

munich_merged.head()

,District,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allach-Untermenzing,80995,48.190034,11.468105,3,Drugstore,Hotel,Trattoria/Osteria,Sporting Goods Shop,Yoga Studio,Farmers Market,Fountain,Food Court,Food & Drink Shop,Food
1,Allach-Untermenzing,80997,48.183434,11.488162,3,Drugstore,Hotel,Trattoria/Osteria,Sporting Goods Shop,Yoga Studio,Farmers Market,Fountain,Food Court,Food & Drink Shop,Food
2,Allach-Untermenzing,80999,48.195994,11.457013,3,Drugstore,Hotel,Trattoria/Osteria,Sporting Goods Shop,Yoga Studio,Farmers Market,Fountain,Food Court,Food & Drink Shop,Food
3,Allach-Untermenzing,81247,48.176884,11.476058,3,Drugstore,Hotel,Trattoria/Osteria,Sporting Goods Shop,Yoga Studio,Farmers Market,Fountain,Food Court,Food & Drink Shop,Food
4,Allach-Untermenzing,81249,48.176884,11.476058,3,Drugstore,Hotel,Trattoria/Osteria,Sporting Goods Shop,Yoga Studio,Farmers Market,Fountain,Food Court,Food & Drink Shop,Food


Finally, let's visualize the resulting clusters

In [56]:
# create map
map_clusters = folium.Map(location=[latitude_munich, longitude_munich], zoom_start=11)

# set color scheme for the clusters
x = np.arange(num_clusters)
ys = [i + x + (i*x)**2 for i in range(num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i**4) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(munich_merged['Latitude'], munich_merged['Longitude'], munich_merged['District'], munich_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

As last step, each cluster shall be examined according to its most frequent venues. Plot all values within a table.

In [57]:
munich_merged.loc[munich_merged['Cluster Labels'] == 0, munich_merged.columns[[1] + list(range(5, munich_merged.shape[1]))]]

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
54,80807,Coffee Shop,German Restaurant,Art Museum,Café,Fast Food Restaurant,Fountain,Food Court,Food & Drink Shop,Food,Flower Shop
55,80809,Coffee Shop,German Restaurant,Art Museum,Café,Fast Food Restaurant,Fountain,Food Court,Food & Drink Shop,Food,Flower Shop
56,80937,Coffee Shop,German Restaurant,Art Museum,Café,Fast Food Restaurant,Fountain,Food Court,Food & Drink Shop,Food,Flower Shop
57,80939,Coffee Shop,German Restaurant,Art Museum,Café,Fast Food Restaurant,Fountain,Food Court,Food & Drink Shop,Food,Flower Shop
72,80687,Pizza Place,Jewelry Store,Café,Hotel,Restaurant,German Restaurant,Yoga Studio,Fast Food Restaurant,Food Court,Food & Drink Shop
73,80689,Pizza Place,Jewelry Store,Café,Hotel,Restaurant,German Restaurant,Yoga Studio,Fast Food Restaurant,Food Court,Food & Drink Shop
74,81241,Pizza Place,Jewelry Store,Café,Hotel,Restaurant,German Restaurant,Yoga Studio,Fast Food Restaurant,Food Court,Food & Drink Shop
75,81243,Pizza Place,Jewelry Store,Café,Hotel,Restaurant,German Restaurant,Yoga Studio,Fast Food Restaurant,Food Court,Food & Drink Shop
76,81245,Pizza Place,Jewelry Store,Café,Hotel,Restaurant,German Restaurant,Yoga Studio,Fast Food Restaurant,Food Court,Food & Drink Shop
77,81247,Pizza Place,Jewelry Store,Café,Hotel,Restaurant,German Restaurant,Yoga Studio,Fast Food Restaurant,Food Court,Food & Drink Shop


In [58]:
munich_merged.loc[munich_merged['Cluster Labels'] == 1, munich_merged.columns[[1] + list(range(5, munich_merged.shape[1]))]]

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,81243,Bavarian Restaurant,Italian Restaurant,Fast Food Restaurant,French Restaurant,Fountain,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market
20,81245,Bavarian Restaurant,Italian Restaurant,Fast Food Restaurant,French Restaurant,Fountain,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market
21,81249,Bavarian Restaurant,Italian Restaurant,Fast Food Restaurant,French Restaurant,Fountain,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market
38,80686,Department Store,German Restaurant,Church,Fried Chicken Joint,French Restaurant,Fountain,Food Court,Food & Drink Shop,Food,Flower Shop
39,80687,Department Store,German Restaurant,Church,Fried Chicken Joint,French Restaurant,Fountain,Food Court,Food & Drink Shop,Food,Flower Shop
40,80689,Department Store,German Restaurant,Church,Fried Chicken Joint,French Restaurant,Fountain,Food Court,Food & Drink Shop,Food,Flower Shop
41,80335,Irish Pub,Gourmet Shop,Men's Store,Bavarian Restaurant,Fruit & Vegetable Store,French Restaurant,Fountain,Food Court,Food & Drink Shop,Food
42,80336,Irish Pub,Gourmet Shop,Men's Store,Bavarian Restaurant,Fruit & Vegetable Store,French Restaurant,Fountain,Food Court,Food & Drink Shop,Food
43,80337,Irish Pub,Gourmet Shop,Men's Store,Bavarian Restaurant,Fruit & Vegetable Store,French Restaurant,Fountain,Food Court,Food & Drink Shop,Food
44,80469,Irish Pub,Gourmet Shop,Men's Store,Bavarian Restaurant,Fruit & Vegetable Store,French Restaurant,Fountain,Food Court,Food & Drink Shop,Food


In [59]:
munich_merged.loc[munich_merged['Cluster Labels'] == 2, munich_merged.columns[[1] + list(range(5, munich_merged.shape[1]))]]

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
101,80335,Italian Restaurant,Food Court,Farmers Market,French Restaurant,Fountain,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market
102,80339,Italian Restaurant,Food Court,Farmers Market,French Restaurant,Fountain,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market


In [60]:
munich_merged.loc[munich_merged['Cluster Labels'] == 3, munich_merged.columns[[1] + list(range(5, munich_merged.shape[1]))]]

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,80995,Drugstore,Hotel,Trattoria/Osteria,Sporting Goods Shop,Yoga Studio,Farmers Market,Fountain,Food Court,Food & Drink Shop,Food
1,80997,Drugstore,Hotel,Trattoria/Osteria,Sporting Goods Shop,Yoga Studio,Farmers Market,Fountain,Food Court,Food & Drink Shop,Food
2,80999,Drugstore,Hotel,Trattoria/Osteria,Sporting Goods Shop,Yoga Studio,Farmers Market,Fountain,Food Court,Food & Drink Shop,Food
3,81247,Drugstore,Hotel,Trattoria/Osteria,Sporting Goods Shop,Yoga Studio,Farmers Market,Fountain,Food Court,Food & Drink Shop,Food
4,81249,Drugstore,Hotel,Trattoria/Osteria,Sporting Goods Shop,Yoga Studio,Farmers Market,Fountain,Food Court,Food & Drink Shop,Food
5,80331,Supermarket,Sporting Goods Shop,Vietnamese Restaurant,Bus Stop,Light Rail Station,Bakery,Yoga Studio,Fast Food Restaurant,Fountain,Food Court
6,80333,Supermarket,Sporting Goods Shop,Vietnamese Restaurant,Bus Stop,Light Rail Station,Bakery,Yoga Studio,Fast Food Restaurant,Fountain,Food Court
7,80335,Supermarket,Sporting Goods Shop,Vietnamese Restaurant,Bus Stop,Light Rail Station,Bakery,Yoga Studio,Fast Food Restaurant,Fountain,Food Court
8,80336,Supermarket,Sporting Goods Shop,Vietnamese Restaurant,Bus Stop,Light Rail Station,Bakery,Yoga Studio,Fast Food Restaurant,Fountain,Food Court
9,80469,Supermarket,Sporting Goods Shop,Vietnamese Restaurant,Bus Stop,Light Rail Station,Bakery,Yoga Studio,Fast Food Restaurant,Fountain,Food Court


In [61]:
munich_merged.loc[munich_merged['Cluster Labels'] == 4, munich_merged.columns[[1] + list(range(5, munich_merged.shape[1]))]]

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,80689,Fountain,Plaza,Gourmet Shop,Yoga Studio,Farmers Market,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market
36,81375,Fountain,Plaza,Gourmet Shop,Yoga Studio,Farmers Market,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market
37,81377,Fountain,Plaza,Gourmet Shop,Yoga Studio,Farmers Market,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market
68,81539,Plaza,Shopping Mall,Organic Grocery,Church,Fast Food Restaurant,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market
69,81541,Plaza,Shopping Mall,Organic Grocery,Church,Fast Food Restaurant,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market
70,81547,Plaza,Shopping Mall,Organic Grocery,Church,Fast Food Restaurant,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market
71,81549,Plaza,Shopping Mall,Organic Grocery,Church,Fast Food Restaurant,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market
103,80336,Plaza,Soup Place,German Restaurant,Pedestrian Plaza,Cocktail Bar,Farmers Market,Food Court,Food & Drink Shop,Food,Flower Shop
104,80337,Plaza,Soup Place,German Restaurant,Pedestrian Plaza,Cocktail Bar,Farmers Market,Food Court,Food & Drink Shop,Food,Flower Shop
105,80469,Plaza,Soup Place,German Restaurant,Pedestrian Plaza,Cocktail Bar,Farmers Market,Food Court,Food & Drink Shop,Food,Flower Shop
